In [1]:
import pickle
import pandas as pd
import numpy as np
import itertools

In [2]:
#load dictionary of matrix
with open('rfam_PPM_family_dic.pickle', 'rb') as afile:
        PPM_dic=pickle.load(afile)
        
with open('rfam_PSSM_family_dic.pickle', 'rb') as afile:
        PSSM_dic=pickle.load(afile)

In [3]:
PPM_2 = {k: PPM_dic[k] for k in list(PPM_dic)[:4]}
PSSM_2 = {k: PSSM_dic[k] for k in list(PSSM_dic)[:4]}

In [4]:
fam=list(PSSM_2.keys())
coppie=list(itertools.combinations(fam,2))

In [30]:
def KLD_fin2(l_finestra, diff, PPM_q, PPM_t, PSSM_q, PSSM_t):
    kld_finestre=[]
    if diff<0: #se la PSSM query e piu piccola della target, scorro sulla target.
        print('diff<0')
        for shift in range(0, -diff):
            kld_finestra=0

            for x in range(0, l_finestra):
                y=x+shift
                x2=x
                tot_x=0
                tot_y=0
                for ch in range(0, 19):
                    tot_x+=(PPM_q.iloc[ch, x2]*(PSSM_q.iloc[ch,x2]-PSSM_t.iloc[ch,y]))
                    tot_y+=(PPM_t.iloc[ch, y]*(PSSM_t.iloc[ch,y]-PSSM_q.iloc[ch,x2]))
                kld_finestra+=(0.5*(tot_x+tot_y))
            kld_finestre.append(kld_finestra)
            
    elif diff>0: #se la PSSM query e piu grande della target, scorro sulla query.
        print('diff>0')
        for shift in range(0, diff):
            kld_finestra=0
            for x in range(0, l_finestra):
                x2=x+shift
                y=x
                tot_x=0
                tot_y=0
                for ch in range(0, 19):
                    tot_x+=(PPM_q.iloc[ch, x2]*(PSSM_q.iloc[ch,x2]-PSSM_t.iloc[ch,y]))
                    tot_y+=(PPM_t.iloc[ch, y]*(PSSM_t.iloc[ch,y]-PSSM_q.iloc[ch,x2]))
                kld_finestra+=(0.5*(tot_x+tot_y))
            kld_finestre.append(kld_finestra)
    else:
        print('diff==0')
        kld_finestra=0
        for x in range(0, l_finestra):
            y=x
            x2=x
            tot_x=0
            tot_y=0
            for ch in range(0, 19):
                tot_x+=(PPM_q.iloc[ch, x2]*(PSSM_q.iloc[ch,x2]-PSSM_t.iloc[ch,y]))
                tot_y+=(PPM_t.iloc[ch, y]*(PSSM_t.iloc[ch,y]-PSSM_q.iloc[ch,x2]))
            kld_finestra+=(0.5*(tot_x+tot_y))
        kld_finestre.append(kld_finestra)
    return kld_finestre 
    

In [ ]:
def KLD_fin(l_finestra, diff, PPM_q, PPM_t, PSSM_q, PSSM_t):
    kld_finestre=[]
    if diff<0: #se la PSSM query e piu piccola della target, scorro sulla target.
        print('diff<0')
        for shift in range(0, -diff):
            kld_finestra=0
            for x in range(0, l_finestra):
                y=x+shift
                tot_x=0
                tot_y=0
                for ch in range(0, 19):
                    tot_x+=(PPM_q.iloc[ch, x]*(PSSM_q.iloc[ch,x]-PSSM_t.iloc[ch,y]))
                    tot_y+=(PPM_t.iloc[ch, y]*(PSSM_t.iloc[ch,y]-PSSM_q.iloc[ch,x]))
                kld_finestra+=(0.5*(tot_x+tot_y))
            kld_finestre.append(kld_finestra)
            
    elif diff>0: #se la PSSM query e piu grande della target, scorro sulla query.
        print('diff>0')
        for shift in range(0, diff):
            kld_finestra=0
            for y in range(0, l_finestra):
                x=y+shift
                tot_x=0
                tot_y=0
                for ch in range(0, 19):
                    tot_x+=(PPM_q.iloc[ch, x]*(PSSM_q.iloc[ch,x]-PSSM_t.iloc[ch,y]))
                    tot_y+=(PPM_t.iloc[ch, y]*(PSSM_t.iloc[ch,y]-PSSM_q.iloc[ch,x]))
                kld_finestra+=(0.5*(tot_x+tot_y))
            kld_finestre.append(kld_finestra)
    else:
        print('diff==0')
        kld_finestra=0
        for y in range(0, l_finestra):
            x=y
            tot_x=0
            tot_y=0
            for ch in range(0, 19):
                tot_x+=(PPM_q.iloc[ch, x]*(PSSM_q.iloc[ch,x]-PSSM_t.iloc[ch,y]))
                tot_y+=(PPM_t.iloc[ch, y]*(PSSM_t.iloc[ch,y]-PSSM_q.iloc[ch,x]))
            kld_finestra+=(0.5*(tot_x+tot_y))
        kld_finestre.append(kld_finestra)

    return kld_finestre 

In [ ]:
d_KLD_family={}
for coppia in coppie:
    print(coppia)
    d_KLD_family[coppia]=[]
    
    
    PSSM_q=pd.DataFrame(PSSM_2[coppia[0]])
    PPM_q=pd.DataFrame(PPM_2[coppia[0]])
    
    PSSM_t=pd.DataFrame(PSSM_2[coppia[1]])
    PPM_t=pd.DataFrame(PPM_2[coppia[1]])

    l_finestra=min(len(PSSM_q.columns.values),len(PSSM_t.columns.values))
    diff=len(PSSM_q.columns.values)-len(PSSM_t.columns.values)
    #print(diff)
    
    kld_fin=KLD_fin(l_finestra, diff, PPM_q, PPM_t, PSSM_q, PSSM_t)
    
    d_KLD_family[coppia]=kld_fin
    print(kld_fin)
        
      

In [27]:
d_KLD_family2={}
for coppia in coppie:
    d_KLD_family2[coppia]=[]
    
    
    PSSM_q=pd.DataFrame(PSSM_2[coppia[0]])
    PPM_q=pd.DataFrame(PPM_2[coppia[0]])
    
    PSSM_t=pd.DataFrame(PSSM_2[coppia[1]])
    PPM_t=pd.DataFrame(PPM_2[coppia[1]])

    l_finestra=min(len(PSSM_q.columns.values),len(PSSM_t.columns.values))
    diff=len(PSSM_q.columns.values)-len(PSSM_t.columns.values)
    #print(diff)
    
    kld_fin=KLD_fin2(l_finestra, diff, PPM_q, PPM_t, PSSM_q, PSSM_t)
    
    d_KLD_family2[coppia]=kld_fin

diff>0
diff>0
diff<0
diff>0
diff<0
diff<0


In [28]:
d_KLD_family

{('RF00001', 'RF00002'): [-101231.61789376815,
  -100713.22016075638,
  -101270.08877240887,
  -102308.45751348486,
  -102589.78217300778,
  -104049.64373507397,
  -106269.72008756573,
  -108610.6918416286,
  -111006.07400546316,
  -110513.96954074322,
  -110898.42812906002,
  -112704.62765813332,
  -110538.71161607359,
  -109379.71029723171,
  -108438.50096091919,
  -109557.31465393715,
  -108067.9605197772,
  -109712.27854116785,
  -110432.14964494492,
  -107842.93668264686,
  -107072.40031411209,
  -106185.25357901686,
  -105714.91189471804],
 ('RF00001', 'RF00003'): [-75535.91901638861,
  -74663.5571078964,
  -73552.16851695729,
  -75452.69230398901,
  -76337.48459451107,
  -77646.09443837588,
  -80494.11984074382,
  -82517.43460512388,
  -84493.8930815559,
  -84406.09242614522,
  -87468.4409530877,
  -90603.59609353782,
  -91245.93135732872,
  -91626.75912716225,
  -89779.20607527708,
  -89470.27090342031,
  -88795.55130442319,
  -86569.8702395785,
  -86736.18990673119,
  -86563.6

In [29]:
d_KLD_family2

{('RF00001', 'RF00002'): [-101231.61789376815,
  -100713.22016075638,
  -101270.08877240887,
  -102308.45751348486,
  -102589.78217300778,
  -104049.64373507397,
  -106269.72008756573,
  -108610.6918416286,
  -111006.07400546316,
  -110513.96954074322,
  -110898.42812906002,
  -112704.62765813332,
  -110538.71161607359,
  -109379.71029723171,
  -108438.50096091919,
  -109557.31465393715,
  -108067.9605197772,
  -109712.27854116785,
  -110432.14964494492,
  -107842.93668264686,
  -107072.40031411209,
  -106185.25357901686,
  -105714.91189471804],
 ('RF00001', 'RF00003'): [-75535.91901638861,
  -74663.5571078964,
  -73552.16851695729,
  -75452.69230398901,
  -76337.48459451107,
  -77646.09443837588,
  -80494.11984074382,
  -82517.43460512388,
  -84493.8930815559,
  -84406.09242614522,
  -87468.4409530877,
  -90603.59609353782,
  -91245.93135732872,
  -91626.75912716225,
  -89779.20607527708,
  -89470.27090342031,
  -88795.55130442319,
  -86569.8702395785,
  -86736.18990673119,
  -86563.6